In [ ]:
## before anything you need to visit the site below and make sure you have a google earth engine account
## this is so you can access Sentinel-1 GRD and Sentinel-2 TOA and SR products, as well as other sensor packages and data types

## visit the below website below to setup an earth engine account, enable a cloud project, and enable the ee API 
## https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup#get-access-to-earth-engine

In [ ]:
import ee
import geemap

In [ ]:
## only need to run this once
## after authenticating with google earth engine you will only need to initialize each session

## https://developers.google.com/earth-engine/guides/auth
ee.Authenticate(auth_mode = 'notebook')

In [ ]:
## init ee cloud project you made during initial setup
ee.Initialize(project = your cloud project) ##enter your project name here as a string to initialize exchanges with ee api

# Some functions for a bit easier mapping
super simple for now, might make them better later

In [ ]:
## Function to add RGB images to the map.
def add_rgb_to_map(image, map_object):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    for band in s2_10m.first().bandNames().getInfo(): ## all images if small enough image collection
        map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')
    # if date[:8] == '20200106': ## can put specific dates here in YYYYMMDD
        # map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')
    # elif date[:8] == '20191224': ## can put specific dates here in YYYYMMDD
        # map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')
    # map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')
    # print(date[:8])


## Function to add spectral indices images to the map.
def add_ind_to_map(image, map_object):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    for band in s2_10m_ind.first().bandNames().getInfo():
        if band =='NDWI':
            map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['00FFFF', '0000FF']}, f'{date}_{band}')
        elif band =='NDVI': 
            map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['FF0000', '008000']}, f'{date}_{band}')
        # if date[:8] == '20200106':
        #     if band =='NDWI':
        #         map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['00FFFF', '0000FF']}, f'{date}_{band}')
        #     elif band =='NDVI': 
        #         map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['FF0000', '008000']}, f'{date}_{band}')
        # elif date[:8] == '20191224':
        #     if band == 'NDWI':
        #         map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['00FFFF', '0000FF']}, f'{date}_{band}')
        #     elif band =='NDVI': 
        #         map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': ['FF0000', '008000']}, f'{date}_{band}')

In [ ]:
## functiont to create three important and popular spectral indices
## ndvi = Normalized Difference Vegetation Index, good for vegetation health and cover
## ndwi = Normalized Difference Water Index, good for identifying water bodies and mositure in surface
def s2_10m_target_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands([ndvi, ndwi])

## collects sentinel-1 GRD (radar, no phase) and Sentinel-2 SR (multispectral, adjusted for top of atmosphere reflectance)
def get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit):
    ## Sentinel-1 ImageCollection
    s1 = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.eq('orbitProperties_pass', orbit))
               .select(['VV', 'VH'])
               .sort('date')
    )

    ## Clip all images in the collection to the AOI
    s1 = s1.map(lambda img: img.clip(aoi))
    ## Sentinel-2 Surface Reflectance Harmonized ImageCollection
    s2_10m = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', s2_cloud_cov))
               .sort('date')
               .select(['B2', 'B3', 'B4', 'B8'])
    )
    ## Clip all images in the collection to the AOI
    s2_10m = s2_10m.map(lambda img: img.clip(aoi))
    ## Apply indices to the Sentinel-2 images
    s2_10m_ind = s2_10m.map(s2_10m_target_indices).select(['NDVI', 'NDWI'])

    return s1, s2_10m, s2_10m_ind


In [ ]:
## fucntion to get the date of each image in the image collection
def get_date(image):
    return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})

In [ ]:
def export_image_to_drive(collection, image, index):
    # Define the description for the export, incorporating the index for uniqueness
    if collection == s2_10m:
        description = f's2_{index}'
    elif collection == s2_10m_ind:
        description = f's2_ind_{index}'
    else:
        description = f's1_{index}'
    
    # Setup the export task
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        region=aoi,  # Make sure the geometry is defined earlier
        fileFormat='GeoTIFF',
        scale = 10
    )
    task.start()
    print(f'Exporting {description} to Drive...')

def export_all_images(collection):
    image_list = collection.toList(collection.size())  # Convert ImageCollection to List
    num_images = image_list.size().getInfo()  # Get the number of images

    if collection == s2_10m:
        for i, date in enumerate(s2_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, image, date[:10])
            # if date[:10] == '2019-12-24' or date[:10] == '2020-01-06': ## can be target dates you are specifically looking for
            #     image = ee.Image(image_list.get(i))  # Get the image from the list
            #     export_image_to_drive(collection, image, date[:-9])  # Export this image

    elif collection == s2_10m_ind:
        for i, date in enumerate(s2_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, image, date[:10])
            # if date[:10] == '2019-12-24' or date[:10] == '2020-01-06': ## can be target dates you are specifically looking for
            #     image = ee.Image(image_list.get(i))  # Get the image from the list
            #     export_image_to_drive(collection, image, date[:-9])  # Export this image
    else:
        for i, date in enumerate(s1_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, image, date[:10])
            # if date[:10] == '2019-12-24' or date[:10] == '2020-01-06': ## can be target dates you are specifically looking for
            #     image = ee.Image(image_list.get(i))  # Get the image from the list
            #     export_image_to_drive(collection, image, date[:-9])  # Export this image

# Get area of interest

In [ ]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='OPENSTREETMAP')
## Add drawing tools
m.add_draw_control()
## Display the map
display(m)

In [ ]:
## Get the drawn features
draw_features = m.draw_features[0]
## Establish ee.Polygon from drawn area of interest to collect imagery
aoi = ee.Geometry.Polygon(draw_features.getInfo()['geometry']['coordinates'][0])

# Get Imagery

In [ ]:
start_date = '2020-06-29' ## start date of search window
end_date = '2020-07-13' ## end date of search window
s2_cloud_cov = 20 ## percentage of clouds in sentinel-2 multispectral imagery, less means you see more surface
orbit = 'ASCENDING' ## orbit for imagery

s1, s2_10m, s2_10m_ind = get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)

In [ ]:
s1_dates = s1.map(get_date)
s2_dates = s2_10m.map(get_date)

# Convert to a list
s1_date_list = s1_dates.aggregate_array('date').getInfo()
s2_date_list = s2_dates.aggregate_array('date').getInfo()

In [ ]:
display(f's1 date list: {s1_date_list}')
display(f's2 date list: {s2_date_list}')

# Map imagery

In [ ]:
Map = geemap.Map()

# Visualize each image in the ImageCollection.
s1_images = s2_10m.toList(s2_10m.size())
for i in range(s2_10m.size().getInfo()):
    image = ee.Image(s1_images.get(i))
    add_rgb_to_map(image, Map)

# Display the map.
Map.addLayerControl(position = 'topright')
Map

In [ ]:
Map_ind = geemap.Map()

# Visualize each image in the ImageCollection.
s1_images = s2_10m_ind.toList(s2_10m_ind.size())
for i in range(s2_10m_ind.size().getInfo()):
    image = ee.Image(s1_images.get(i))
    add_ind_to_map(image, Map_ind)

# Display the map.
Map_ind.addLayerControl(position = 'topright')
Map_ind

# Export imagery to google drive

In [ ]:
# Call the function to start the export
for col in [s1, s2_10m, s2_10m_ind]:
    export_all_images(col)